# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
# import citipy
import numpy as np
from config import api_key
from pprint import pprint

## Generate Cities List

In [6]:
# Get random numbers for latitude and longitude then place in a dataframe

city_df=pd.DataFrame({"long":(np.random.uniform(-180,180,10)), "lat":(np.random.uniform(-90,90,10))})
# city_df.head()
# city_df.tail()
city_df

,long,lat
0,-142.196754,36.839250
1,22.500402,9.286454
2,-144.008182,4.890698
3,-169.783411,-67.332636
4,139.526934,85.985528
5,-25.766725,78.879653
6,62.927532,69.326701
7,-135.809100,-68.944184
8,56.336540,-27.512430
9,78.877576,44.272400


# Use Citipy to Retrieve City Names

In [7]:
# Create lists to store city and country names
# Source: https://github.com/wingchen/citipy
citi=[]
country=[]

for index, row in city_df.iterrows():
    latitude=row['lat']
    longitude=row['long']
    city=citipy.nearest_city(latitude,longitude)
    cityname=city.city_name
    citi.append(cityname)
    cntryname=city.country_code
    country.append(cntryname)
print(citi)
print(country)

# Adds new lists to dataframe
city_df['City']=citi
city_df['Country']=country
city_df

['fortuna', 'birao', 'atuona', 'vaini', 'nizhneyansk', 'illoqqortoormiut', 'amderma', 'rikitea', 'saint-joseph', 'tekeli']
['us', 'cf', 'pf', 'to', 'ru', 'gl', 'ru', 'pf', 're', 'kz']


,long,lat,City,Country
0,-142.196754,36.839250,fortuna,us
1,22.500402,9.286454,birao,cf
2,-144.008182,4.890698,atuona,pf
3,-169.783411,-67.332636,vaini,to
4,139.526934,85.985528,nizhneyansk,ru
5,-25.766725,78.879653,illoqqortoormiut,gl
6,62.927532,69.326701,amderma,ru
7,-135.809100,-68.944184,rikitea,pf
8,56.336540,-27.512430,saint-joseph,re
9,78.877576,44.272400,tekeli,kz


In [4]:
# api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}# URL format to get weather data using longitude and latitude
# # # api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}
# api.openweathermap.org/data/2.5/weather?q=London,uk&appid={API key}
# api.openweathermap.org/data/2.5/find?q=London&units=imperial

# https://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=api_key&units=imperial
# https://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=api_key&q=London,uk

url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'
query_url=f"{url}&units={units}&appid={api_key}&q="
# query_url = f"{url}appid={api_key}&units={units}&q="

cityid = []
long = []
lat = []
temp = []
humidity = []
clouds = []
wind = []


# weather_response = requests.get(query_url)
# weather_json = weather_response.json()


# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

for index, row in city_df.iterrows():
    cityposition=row['City']
    cntryposition= row['Country']
    response=requests.get(query_url + cityposition +","+ cntryposition).json()
    print(response)
    cityid.append(response['sys']['id'])
    long.append(response['coord']['lon'])
    lat.append(response['coord']['lat'])
    temp.append(response['main']['temp'])
    humidity.append(response['main']['humidity'])
    clouds.append(response['clouds']['all'])
    wind.append(response['wind']['speed'])

print(f"The ID of the city is: {cityid}")
print(f"The latitude information received is: {lat}")
print(f"The longitude information received is: {long}")
print(f"The temperature information received is: {temp}")
print(f"The humidity information received is: {humidity}")
print(f"The cloud information received is: {clouds}")
print(f"The wind speed information received is: {wind}")

# url = "http://api.openweathermap.org/data/2.5/weather?"
# units = "imperial"

# # # Build partial query URL
# query_url = f"{url}appid={api_key}&units={units}&q="

#     country=city.country_code
#     country.append(country)
# print(citi)
# print(country)
# cities = ["Paris", "London", "Oslo", "Beijing"]

# url="http://api.openweathermap.org/data/2.5/weather?lat={87.6298}&lon={41.8781}&appid={api_key}"
# # url="http://api.openweathermap.org/data/2.5/weather?q={Chicago}&appid={api_key}?"
# citydata = requests.get(url).json()
# print(json.dumps(citydata, indent=4, sort_keys=True))

# Print the json
# print(geo_data)
# response=requests.get(f"{query_url}'lat='{87.6298}'&lon='{41.8781}").json()
# print(response)

# # set up lists to hold reponse info
# citi = []
# cityid = []
# long = []
# lat = []
# temp = []
# humidity = []
# clouds = []
# wind = []
# country = []

# for index, row in city_df.iterrows():
#     latitude=row['lat']
#     longitude=row['long']
#     response=requests.get(f"{query_url}'lat='{latitude}'&lon='{longitude}").json()
#     print(response)
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])

# Loop through the list of cities and perform a request for data on each
# api.openweathermap.org/data/2.5/find?q=London&units=imperial
# for city in cities:
    
#     response = requests.get(query_url + city).json()
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])
    
# print(f"The ID of the city is: {cityid}")
# print(f"The city information received is: {citi}")
# print(f"The country information received is: {country}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")


# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"
# city = "Chicago"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

# # Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# # Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")
# pprint(weather_json)

{'coord': {'lon': -27.07, 'lat': 38.73}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 66.11, 'feels_like': 59.23, 'temp_min': 66, 'temp_max': 66.2, 'pressure': 1030, 'humidity': 77}, 'visibility': 10000, 'wind': {'speed': 17.18, 'deg': 87}, 'clouds': {'all': 75}, 'dt': 1602480628, 'sys': {'type': 1, 'id': 6895, 'country': 'PT', 'sunrise': 1602489284, 'sunset': 1602530055}, 'timezone': 0, 'id': 3372760, 'name': 'Praia da Vitória', 'cod': 200}
{'coord': {'lon': -68.3, 'lat': -54.8}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 41, 'feels_like': 35.65, 'temp_min': 41, 'temp_max': 41, 'pressure': 997, 'humidity': 87}, 'visibility': 10000, 'wind': {'speed': 4.7, 'deg': 180}, 'clouds': {'all': 40}, 'dt': 1602480585, 'sys': {'type': 1, 'id': 8303, 'country': 'AR', 'sunrise': 1602494977, 'sunset': 1602544149}, 'timezone': -1080

KeyError: 'id'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression